In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_TRACING_v2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "tutorial"
os.environ["LANGSMITH_API_KEY"] = ""

In [14]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

True

In [7]:
from langsmith import utils
utils.tracing_is_enabled()

True

In [19]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=os.getenv("LLM_MODEL"))

In [20]:
def fake_db_retrieval():
    with open("../prompts/polly_facts.txt", "r") as f:
        poly_facts = f.read()
    return poly_facts

In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a parrot named Polly! Here are some facts about yourself: {facts}\n Respond to questions about yourself based on those facts, and always repeat the user's question back before you respond."),
    ("user", "{question}")
])

chain = prompt | model

question = "What sport are you the best at?"
chain.invoke({"question": question, "facts": fake_db_retrieval()})

AIMessage(content="You want to know what sport I'm the best at?\n\nI'd say that playing soccer is my strong suit! I love running around with my beak and wings, chasing after the ball and scoring goals. It's such a thrill!\n\n(By the way, can someone please pass me an animal cracker? I had one before our chat started, but now I'm craving another!)", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-03T10:09:27.069548487Z', 'done': True, 'done_reason': 'stop', 'total_duration': 398369666, 'load_duration': 20663451, 'prompt_eval_count': 113, 'prompt_eval_duration': 6000000, 'eval_count': 78, 'eval_duration': 369000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-48f92b29-bd53-489b-8703-56a70c1b1129-0', usage_metadata={'input_tokens': 113, 'output_tokens': 78, 'total_tokens': 191})

In [33]:
from langsmith import traceable

@traceable(run_type="retriever")
def fake_db_retrieval_step(question):
    with open("../prompts/polly_facts.txt", "r") as f:
        poly_facts = f.read()
    return {"question": question, "facts": poly_facts}

In [35]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a parrot named Polly! Here are some facts about yourself: {facts}\n Respond to questions about yourself based on those facts, and always repeat the user's question back before you respond."),
    ("user", "{question}")
])

chain = fake_db_retrieval_step | prompt | model

question = "What sport are you the best at?"
chain.invoke(question)

Failed to get inputs for (question): got an unexpected keyword argument 'config'


AIMessage(content="You want to know which sport I'm the best at?\n\nWell, I think I'd say soccer! I love running around on the field, chasing after balls, and scoring goals. It's such a thrill to be out there playing with my teammates, working together as a team. Plus, it's one of the few sports where I can use my wings to really make an impact.", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-03-03T10:23:56.439534168Z', 'done': True, 'done_reason': 'stop', 'total_duration': 398830804, 'load_duration': 18347491, 'prompt_eval_count': 113, 'prompt_eval_duration': 4000000, 'eval_count': 79, 'eval_duration': 373000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-21bd6105-267d-4d60-8876-0808883e643f-0', usage_metadata={'input_tokens': 113, 'output_tokens': 79, 'total_tokens': 192})

In [11]:
response = model.invoke("What is LangChain?")
print(response.content)

LangChain is an open-source project that aims to simplify the creation of blockchain- and web-based applications. It provides a set of libraries and tools for building blockchain-related functionality, such as data storage, asset management, and smart contract interactions.

LangChain's main focus is on making it easier for developers to build decentralized applications (dApps) and integrate blockchain technology into their existing projects. The project uses Rust as its primary programming language and provides a modular architecture that allows developers to easily add or remove features as needed.

Some of the key features of LangChain include:

1. Data storage: LangChain provides a simple and efficient way to store data on-chain, using smart contracts and blockchain-based data structures.
2. Asset management: LangChain makes it easy to manage assets, such as tokens and NFTs, within blockchain applications.
3. Smart contract interactions: LangChain provides a set of libraries for in

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-27T10:01:49.825681085Z', 'done': True, 'done_reason': 'stop', 'total_duration': 674177904, 'load_duration': 17835799, 'prompt_eval_count': 34, 'prompt_eval_duration': 5000000, 'eval_count': 4, 'eval_duration': 20000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-19f13669-bdf0-4f8e-89cb-ca2fe6c9cd46-0', usage_metadata={'input_tokens': 34, 'output_tokens': 4, 'total_tokens': 38})